In [1]:
import sys
sys.path.append("../src")
import os

import pandas as pd
import numpy as np
from etf_transformations import *
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.preprocessing import FunctionTransformer

In [2]:
df = pd.read_csv("../data/preprocessed/final_etf_data/weekends_aggregated_embedding/XLK_v4.csv")
df = sign_next_day(df)
df = drop_sign_and_return(df)


In [3]:
df.head(10)

,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,63.310219,0.258843,0.683649,0.057507,6.0,0.201336,-0.040082,-0.042018,0.012110,...,-0.006043,-0.045790,-0.053740,0.008389,0.029547,-0.061633,0.056060,0.029845,1.0,-1.0
1,2018-03-21,62.929611,0.079694,0.891288,0.029018,3.0,0.050676,-0.055223,-0.005217,0.019322,...,0.039590,-0.018541,-0.059512,0.014814,0.063403,-0.086791,0.030065,0.011303,1.0,-1.0
2,2018-03-22,61.305096,0.348509,0.226298,0.425194,5.0,-0.076685,-0.029817,0.008005,0.017561,...,0.002415,0.003987,-0.066090,0.015980,0.000298,-0.078723,0.040454,0.003072,1.0,-1.0
3,2018-03-23,59.671288,0.241439,0.482122,0.276438,5.0,-0.034999,-0.015262,-0.058099,-0.005390,...,-0.027297,-0.014804,0.027459,-0.004768,0.018634,-0.081501,-0.006712,0.049573,1.0,1.0
4,2018-03-26,61.936344,0.234637,0.625601,0.139762,6.0,0.094875,-0.029753,-0.008020,0.026985,...,-0.016723,-0.013599,-0.027924,0.006294,0.021656,-0.090146,0.043457,0.037637,1.0,-1.0
5,2018-03-27,59.949760,0.075072,0.531589,0.393340,7.0,-0.318268,-0.059012,-0.016827,0.007342,...,0.001650,-0.022806,-0.045531,-0.004143,0.051210,-0.061163,0.044529,0.055805,1.0,-1.0
6,2018-03-28,59.550602,0.310788,0.477132,0.212080,9.0,0.098708,-0.038110,-0.029604,0.028321,...,-0.020209,-0.002886,-0.029602,0.028669,-0.007364,-0.095580,-0.004168,0.057462,1.0,1.0
7,2018-03-29,60.729534,0.370141,0.613385,0.016474,3.0,0.353667,-0.021918,0.029772,0.047880,...,0.000990,-0.038287,-0.058590,0.009561,0.067820,-0.044205,0.060755,0.026523,1.0,-1.0
8,2018-04-02,59.281403,0.134719,0.351496,0.513785,10.0,-0.379066,-0.038686,-0.003821,0.047725,...,0.001952,-0.002385,-0.047809,0.009345,0.023480,-0.090401,0.017328,0.019723,1.0,1.0
9,2018-04-03,59.866215,0.196441,0.353618,0.449942,9.0,-0.253501,-0.014222,-0.011880,0.029031,...,-0.006882,-0.027397,-0.056596,0.009188,0.032312,-0.081563,0.019184,0.024981,1.0,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Columns: 393 entries, Date to Sign_next_day
dtypes: float64(392), object(1)
memory usage: 1.8+ MB


In [5]:
df.describe()

,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,emb_3,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
count,587.000000,575.000000,575.000000,575.000000,575.000000,575.000000,575.000000,575.000000,575.000000,575.000000,...,575.000000,575.000000,575.000000,575.000000,575.000000,575.000000,575.000000,575.000000,575.0,586.000000
mean,74.320264,0.242740,0.461673,0.295587,5.227826,-0.052847,-0.021390,-0.017555,0.033040,-0.020357,...,-0.005077,-0.024438,-0.032785,0.011875,0.024896,-0.072660,0.020447,0.029190,1.0,0.134812
std,11.205095,0.167608,0.189443,0.199488,2.959999,0.316049,0.026629,0.025375,0.027444,0.025041,...,0.025741,0.024305,0.026145,0.023251,0.027367,0.027932,0.028318,0.022447,0.0,0.990855
min,54.094036,0.011777,0.028419,0.007281,1.000000,-0.930456,-0.155710,-0.119959,-0.071220,-0.115616,...,-0.083123,-0.105135,-0.166476,-0.061174,-0.075696,-0.164869,-0.078937,-0.055427,1.0,-1.000000
25%,65.923977,0.104693,0.342161,0.150365,3.000000,-0.248600,-0.037207,-0.033760,0.016074,-0.035910,...,-0.019940,-0.038050,-0.046953,-0.002823,0.007233,-0.090287,0.003629,0.016198,1.0,-1.000000
50%,71.749329,0.225908,0.469282,0.281655,5.000000,-0.042757,-0.022115,-0.017549,0.033929,-0.021073,...,-0.004820,-0.024389,-0.033380,0.011175,0.024277,-0.072845,0.020288,0.028807,1.0,1.000000
75%,81.656147,0.332607,0.576060,0.410246,7.000000,0.135948,-0.005548,-0.001878,0.050306,-0.004839,...,0.009515,-0.008235,-0.018106,0.026877,0.042496,-0.056267,0.039652,0.044746,1.0,1.000000
max,103.656654,0.949424,0.938253,0.946362,20.000000,0.927268,0.108407,0.067286,0.122448,0.055278,...,0.106373,0.091001,0.060326,0.104041,0.112662,0.048366,0.141766,0.107768,1.0,1.000000


In [6]:
df.isna().sum()

Date                 0
Price                0
avg_positive_XLK    12
avg_neutral_XLK     12
avg_negative_XLK    12
                    ..
emb_381             12
emb_382             12
emb_383             12
is_trading_day      12
Sign_next_day        1
Length: 393, dtype: int64

In [7]:
df.nunique()

Date                587
Price               568
avg_positive_XLK    575
avg_neutral_XLK     575
avg_negative_XLK    575
                   ... 
emb_381             575
emb_382             575
emb_383             575
is_trading_day        1
Sign_next_day         3
Length: 393, dtype: int64

### defining target - I want to assign:

class 1 to sign_next_day ==1

and class 0 to sign_next_day ∈ {0, -1}

In [8]:
df = df.dropna(subset=["Sign_next_day"])
df["Sign_next_day"] = (df["Sign_next_day"] == 1).astype(int)
df.head()

,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,63.310219,0.258843,0.683649,0.057507,6.0,0.201336,-0.040082,-0.042018,0.012110,...,-0.006043,-0.045790,-0.053740,0.008389,0.029547,-0.061633,0.056060,0.029845,1.0,0
1,2018-03-21,62.929611,0.079694,0.891288,0.029018,3.0,0.050676,-0.055223,-0.005217,0.019322,...,0.039590,-0.018541,-0.059512,0.014814,0.063403,-0.086791,0.030065,0.011303,1.0,0
2,2018-03-22,61.305096,0.348509,0.226298,0.425194,5.0,-0.076685,-0.029817,0.008005,0.017561,...,0.002415,0.003987,-0.066090,0.015980,0.000298,-0.078723,0.040454,0.003072,1.0,0
3,2018-03-23,59.671288,0.241439,0.482122,0.276438,5.0,-0.034999,-0.015262,-0.058099,-0.005390,...,-0.027297,-0.014804,0.027459,-0.004768,0.018634,-0.081501,-0.006712,0.049573,1.0,1
4,2018-03-26,61.936344,0.234637,0.625601,0.139762,6.0,0.094875,-0.029753,-0.008020,0.026985,...,-0.016723,-0.013599,-0.027924,0.006294,0.021656,-0.090146,0.043457,0.037637,1.0,0


In [9]:
df.shape

(586, 393)

In [10]:
df.isna().sum()

Date                 0
Price                0
avg_positive_XLK    12
avg_neutral_XLK     12
avg_negative_XLK    12
                    ..
emb_381             12
emb_382             12
emb_383             12
is_trading_day      12
Sign_next_day        0
Length: 393, dtype: int64

In [11]:
df.head()

,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,63.310219,0.258843,0.683649,0.057507,6.0,0.201336,-0.040082,-0.042018,0.012110,...,-0.006043,-0.045790,-0.053740,0.008389,0.029547,-0.061633,0.056060,0.029845,1.0,0
1,2018-03-21,62.929611,0.079694,0.891288,0.029018,3.0,0.050676,-0.055223,-0.005217,0.019322,...,0.039590,-0.018541,-0.059512,0.014814,0.063403,-0.086791,0.030065,0.011303,1.0,0
2,2018-03-22,61.305096,0.348509,0.226298,0.425194,5.0,-0.076685,-0.029817,0.008005,0.017561,...,0.002415,0.003987,-0.066090,0.015980,0.000298,-0.078723,0.040454,0.003072,1.0,0
3,2018-03-23,59.671288,0.241439,0.482122,0.276438,5.0,-0.034999,-0.015262,-0.058099,-0.005390,...,-0.027297,-0.014804,0.027459,-0.004768,0.018634,-0.081501,-0.006712,0.049573,1.0,1
4,2018-03-26,61.936344,0.234637,0.625601,0.139762,6.0,0.094875,-0.029753,-0.008020,0.026985,...,-0.016723,-0.013599,-0.027924,0.006294,0.021656,-0.090146,0.043457,0.037637,1.0,0


In [12]:
df_catboost = df.copy()
df_catboost.head()

,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,63.310219,0.258843,0.683649,0.057507,6.0,0.201336,-0.040082,-0.042018,0.012110,...,-0.006043,-0.045790,-0.053740,0.008389,0.029547,-0.061633,0.056060,0.029845,1.0,0
1,2018-03-21,62.929611,0.079694,0.891288,0.029018,3.0,0.050676,-0.055223,-0.005217,0.019322,...,0.039590,-0.018541,-0.059512,0.014814,0.063403,-0.086791,0.030065,0.011303,1.0,0
2,2018-03-22,61.305096,0.348509,0.226298,0.425194,5.0,-0.076685,-0.029817,0.008005,0.017561,...,0.002415,0.003987,-0.066090,0.015980,0.000298,-0.078723,0.040454,0.003072,1.0,0
3,2018-03-23,59.671288,0.241439,0.482122,0.276438,5.0,-0.034999,-0.015262,-0.058099,-0.005390,...,-0.027297,-0.014804,0.027459,-0.004768,0.018634,-0.081501,-0.006712,0.049573,1.0,1
4,2018-03-26,61.936344,0.234637,0.625601,0.139762,6.0,0.094875,-0.029753,-0.008020,0.026985,...,-0.016723,-0.013599,-0.027924,0.006294,0.021656,-0.090146,0.043457,0.037637,1.0,0


In [13]:
df_catboost["Date"] = pd.to_datetime(df_catboost["Date"]).dt.normalize()
df_catboost = df_catboost.sort_values("Date").reset_index(drop=True)

df_catboost["no_news"] = df_catboost["n_XLK"].isna().astype(int) # a day where there were no sector specific news
df_catboost.head(10)


,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,no_news
0,2018-03-20,63.310219,0.258843,0.683649,0.057507,6.0,0.201336,-0.040082,-0.042018,0.012110,...,-0.045790,-0.053740,0.008389,0.029547,-0.061633,0.056060,0.029845,1.0,0,0
1,2018-03-21,62.929611,0.079694,0.891288,0.029018,3.0,0.050676,-0.055223,-0.005217,0.019322,...,-0.018541,-0.059512,0.014814,0.063403,-0.086791,0.030065,0.011303,1.0,0,0
2,2018-03-22,61.305096,0.348509,0.226298,0.425194,5.0,-0.076685,-0.029817,0.008005,0.017561,...,0.003987,-0.066090,0.015980,0.000298,-0.078723,0.040454,0.003072,1.0,0,0
3,2018-03-23,59.671288,0.241439,0.482122,0.276438,5.0,-0.034999,-0.015262,-0.058099,-0.005390,...,-0.014804,0.027459,-0.004768,0.018634,-0.081501,-0.006712,0.049573,1.0,1,0
4,2018-03-26,61.936344,0.234637,0.625601,0.139762,6.0,0.094875,-0.029753,-0.008020,0.026985,...,-0.013599,-0.027924,0.006294,0.021656,-0.090146,0.043457,0.037637,1.0,0,0
5,2018-03-27,59.949760,0.075072,0.531589,0.393340,7.0,-0.318268,-0.059012,-0.016827,0.007342,...,-0.022806,-0.045531,-0.004143,0.051210,-0.061163,0.044529,0.055805,1.0,0,0
6,2018-03-28,59.550602,0.310788,0.477132,0.212080,9.0,0.098708,-0.038110,-0.029604,0.028321,...,-0.002886,-0.029602,0.028669,-0.007364,-0.095580,-0.004168,0.057462,1.0,1,0
7,2018-03-29,60.729534,0.370141,0.613385,0.016474,3.0,0.353667,-0.021918,0.029772,0.047880,...,-0.038287,-0.058590,0.009561,0.067820,-0.044205,0.060755,0.026523,1.0,0,0
8,2018-04-02,59.281403,0.134719,0.351496,0.513785,10.0,-0.379066,-0.038686,-0.003821,0.047725,...,-0.002385,-0.047809,0.009345,0.023480,-0.090401,0.017328,0.019723,1.0,1,0
9,2018-04-03,59.866215,0.196441,0.353618,0.449942,9.0,-0.253501,-0.014222,-0.011880,0.029031,...,-0.027397,-0.056596,0.009188,0.032312,-0.081563,0.019184,0.024981,1.0,1,0


In [14]:
df_catboost = df_catboost.drop(columns=["Date"])
df_catboost.head(1)

,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,emb_3,...,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,no_news
0,63.310219,0.258843,0.683649,0.057507,6.0,0.201336,-0.040082,-0.042018,0.01211,-0.008352,...,-0.04579,-0.05374,0.008389,0.029547,-0.061633,0.05606,0.029845,1.0,0,0


In [15]:
original_features = [
    'Price',
    'avg_positive_XLK',
    'avg_neutral_XLK', 
    'avg_negative_XLK',
    'n_XLK',
    'sent_index_XLK',
    'is_trading_day'
]


embedding_features = [f'emb_{i}' for i in range(384)]

target = 'Sign_next_day'
df_clean = df_catboost.dropna(subset=[target, 'is_trading_day']).copy()


for emb_col in embedding_features:
    df_clean[emb_col] = df_clean[emb_col].fillna(0)


df_clean['avg_positive_XLK'] = df_clean['avg_positive_XLK'].fillna(0)
df_clean['avg_neutral_XLK'] = df_clean['avg_neutral_XLK'].fillna(1)
df_clean['avg_negative_XLK'] = df_clean['avg_negative_XLK'].fillna(0)
df_clean['n_XLK'] = df_clean['n_XLK'].fillna(0)
df_clean['sent_index_XLK'] = df_clean['sent_index_XLK'].fillna(0)

print(f"Clean dataset size: {len(df_clean)} rows")


Clean dataset size: 574 rows


In [16]:
X = df_clean[original_features + embedding_features]
y = df_clean[target]

#y = (y == 1).astype(int)

print(f"Target distribution: {y.value_counts().to_dict()}")

Target distribution: {1: 325, 0: 249}


In [17]:
embedding_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95, random_state=12))  # 95% variance
])


original_pipeline = Pipeline([
    ('passthrough', FunctionTransformer())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('original', original_pipeline, original_features),
        ('embeddings', embedding_pipeline, embedding_features)
    ],
    remainder='drop'
)

In [18]:
catboost = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    train_dir="cb_tmp",
    allow_writing_files=False
)

full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', catboost)
])


param_distributions = {
    'preprocessor__embeddings__pca__n_components': [0.90, 0.95, 0.98, 50, 75, 100],
    'classifier__depth': randint(3, 7),
    'classifier__learning_rate': uniform(0.01, 0.19),
    'classifier__iterations': randint(50, 500),
    'classifier__l2_leaf_reg': uniform(1, 14),
    'classifier__border_count': randint(32, 128),
    'classifier__random_strength': uniform(0.5, 5),
    'classifier__min_data_in_leaf': randint(5, 30),
    'classifier__rsm': uniform(0.6, 0.4),
    'classifier__boosting_type': ['Ordered', 'Plain'],
    'classifier__bootstrap_type': ['Bernoulli', 'MVS'],
    'classifier__subsample': uniform(0.6, 0.4),
}

random_search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_distributions,
    n_iter=40,
    cv=5,
    scoring='roc_auc',
    random_state=12,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)


random_search.fit(X, y)

print(f"Best AUC: {random_search.best_score_:.4f}")
print(f"Best parameters:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")



Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best AUC: 0.5338
Best parameters:
  classifier__boosting_type: Plain
  classifier__bootstrap_type: MVS
  classifier__border_count: 89
  classifier__depth: 4
  classifier__iterations: 105
  classifier__l2_leaf_reg: 3.6015975511453227
  classifier__learning_rate: 0.06953914285839076
  classifier__min_data_in_leaf: 19
  classifier__random_strength: 0.5521920104736057
  classifier__rsm: 0.8389835218556493
  classifier__subsample: 0.6338554081408485
  preprocessor__embeddings__pca__n_components: 0.98


In [19]:
best_preprocessor = random_search.best_estimator_.named_steps['preprocessor']
pca = best_preprocessor.named_transformers_['embeddings'].named_steps['pca']

print(f"Number of prncipals components kept: {pca.n_components_}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_.sum():.4f}")
print(f"Original embedding dimensions (384) were reduced to {pca.n_components_}")


Number of prncipals components kept: 217
Explained variance ratio: 0.9800
Original embedding dimensions (384) were reduced to 217
